In [ ]:
import numpy as np
import cv2
import json
from PIL import Image
import pandas as pd
import os, glob
import random
import itertools
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

##################################################################################
##########       VISUALISATION TOOLS FOR BOUNDING BOXES AND LABELS   #############
#################################################################################
DIRECTORY_PATH_FOR_IMAGES_BDG_BXS  = "datasets/json/images"

def draw_rectangle(filename,image,rectangle, center_with_size=False, color=[0, 255, 0]):
        if center_with_size:
            cx, cy, w, h = rectangle
            left, right = int(cx - w/2), int(cx + w/2)
            top, bottom = int(cy - h/2), int(cy + h/2)
        else:
            left, top, right, bottom = rectangle
            img = Image.open(filename)
            w, h = img.size
            if w > 650:
                k = 10
            else:
                k = 2
        return cv2.rectangle(image,(left,top),(right, bottom),color,int(k))

    
def draw_bounding_box(filename,image, bound_box, center_with_size=True, color=(0,255,0)):
    bb_denormalized = bound_box#denormalize_box(bound_box, image.shape[:2])
    result = draw_rectangle(filename,image, bb_denormalized, center_with_size=center_with_size, color=color)
    return result

    
def load_image(filename, flags=-1):
    if not os.path.exists(filename):
        #print (\"file {0} not exists\".format(filename))
        return None
    return cv2.imread(filename, flags)
    
def parse_json_annotation(filename):
        
        filename = filename.split("images")[0]+"annotations"+ filename.split("images")[1].split(".")[0]+".json"  
        bdn_bxs = []
        f= open(filename)
        for line in f:
            line = line.replace("'", '"')
            my_dict = json.loads(line)
            for obj in my_dict["objects"]:
                bdn_bxs.append(obj["bounding_box"])
            return  bdn_bxs
def parse_json_for_imdb_wiki(filename):
        """
        Definition: Parses json annotation file to extract bounding box coordintates, gender, age.
        """
        filename = filename.split("images")[0]+"annotations"+ filename.split("images")[1].split(".")[0]+".json"  

        bdn_bxs = []
        gender = ""
        age = ""
        f= open(filename)
        for line in f:
            line = line.replace("'", '"')
            my_dict = json.loads(line)
            for obj in my_dict["objects"]:
                bdn_bxs.append(obj["bounding_box"])
                gender = obj["gender"]
                age = obj["age"]
            
        return  bdn_bxs, gender, age

def show_bound_box(filename, imdb_wiki):
    
    image = load_image(filename, cv2.IMREAD_COLOR)
    if image is None:
        return
    if imdb_wiki == False:
        coords = parse_json_annotation(filename)
        
        for bound_box in coords:
            result = draw_bounding_box(filename,image, bound_box, center_with_size=False)
        plt.imshow(result)
        plt.show()
    if imdb_wiki == True:
        coords, gender, age = parse_json_for_imdb_wiki(filename)
        for bound_box in coords:
            result = draw_bounding_box(filename,image, bound_box, center_with_size=False)
        
        plt.title("{}, {}".format(int(age),"M" if float(gender)>0.5 else "NAN" if gender == "nan"  else "F"))
        plt.imshow(result)
        plt.show()

def visualize_bnd_boxes_for_all_images(directory_path, imdb_wiki = False):
    
    for filename in glob.glob(os.path.join(directory_path, "*.*")):
        show_bound_box (filename, imdb_wiki)
        
visualize_bnd_boxes_for_all_images(DIRECTORY_PATH_FOR_IMAGES_BDG_BXS,False)